In [1]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [2]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMWQ4MGI4MGYtOGE4Yi00ZGRjLWIxNWItN2E3NDdlNzJlZGFmIiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJyZWNpcGllbnRLZXlzIjogWyJFTjhuWkhXbVFodlBtNnJTcm11V01lYXlCZWZ3YURIUTlvcHBrYU5MWjZoQyJdLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly9hZ2VudDEubG9jYWxob3N0In0=


In [3]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMjY3NWIwMWMtODY2Yi00YWU0LTgyOWQtZDA4NjgxODA0ZjIyIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyJCbVFRNWMyZVhMOW94ZVJNajJuTnBrQzNLZG1odkNIU2txblVQYW1CcG9RayJdLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ==


In [4]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '2675b01c-866b-4ae4-829d-d08681804f22',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['BmQQ5c2eXL9oxeRMj2nNpkC3KdmhvCHSkqnUPamBpoQk'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [5]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 YKPzwzPMxcDDcBBNKqrYZZypLR4HuB5tySwx1aCBTthFRbJAe3ypUaSxf3ajBS7DpB5GNtukZxRLKFjajd96yZj

Public key: 
 DcCLBqoBQopizFRNfbiLT65zdLttzqCycwFcVZ1SPWJd


**Sending a connection request to acapy**

In [6]:
   
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [7]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [8]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:MWMFdJCZkX738paNdnA6bD#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:MWMFdJCZkX738paNdnA6bD',
             'publicKey': [{'controller': 'did:sov:MWMFdJCZkX738paNdnA6bD',
                            'id': 'did:sov:MWMFdJCZkX738paNdnA6bD#1',
                            'publicKeyBase58': 'CBEfywby9FKjMCJrB585GQ4AJf8gHmXZZtKnaqZomq87',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:MWMFdJCZkX738paNdnA6bD;indy',
                          'priority': 0,
                          'recipientKeys': ['CBEfywby9FKjMCJrB585GQ4AJf8gHmXZZtKnaqZomq87'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'\x1b\x01\xbaH]\x0f-\xec\xdd\x9a\x94\xd3\xb7P&8\xcb \xe7q

### FEATURE DISCOVERY

In [17]:
# message = util.buildMessage(
#     "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
#     query="*"
# )
# response = util.sendMessage(message, connections[0])
# pp.pprint(response)

### Notes

* To receive invitation use "new connections" protocol "receive invitation"
* To create invitation use create invitation from "invitations" protocol

### Done
* create inv, receive
* basic messaging sending, get list of messages etc.
* dids

In [11]:
response = util.connectAgents(connections[0], connections[1])
pp.pprint(response)

{'@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/connection', '@id': '72da5a5f-e492-4c20-8c2c-6dbe12ec1e0b', 'state': 'pending', 'label': 'ConnectionBetweenAgents', 'connection_id': '431acade-ed1c-4084-83cd-4084bdbcc2c0', 'my_did': '4M6tANeLSqzAY6pU1x68Zg', 'raw_repr': {'invitation_key': 'GmEA8nDqukbRofaXDSuRCE2UyLA7heUzxnxdTHwKHnYm', 'updated_at': '2020-06-02 10:49:22.929574Z', 'state': 'request', 'accept': 'auto', 'routing_state': 'none', 'invitation_mode': 'once', 'initiator': 'external', 'connection_id': '431acade-ed1c-4084-83cd-4084bdbcc2c0', 'my_did': '4M6tANeLSqzAY6pU1x68Zg', 'request_id': '00624512-b504-4d07-a10e-8b56cecba699', 'created_at': '2020-06-02 10:49:22.899715Z', 'their_label': 'ConnectionBetweenAgents'}}


In [22]:
message = util.buildMessage(
    "https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/get-list",
)
connection_list = util.sendMessage(message, connections[0])
pp.pprint(connection_list['connections'])

[{'connection_id': '8b217d00-aaba-4db2-bff4-8b6616345048',
  'label': 'Client',
  'my_did': 'WwqD8U9uUPHz8ZhVFabXVB',
  'raw_repr': {'accept': 'auto',
               'connection_id': '8b217d00-aaba-4db2-bff4-8b6616345048',
               'created_at': '2020-06-02 10:34:30.442943Z',
               'initiator': 'self',
               'invitation_key': 'AGUEpLAHkUFEQpUi3ALsSBABHhZxh3FWKtYrcf7BcjgR',
               'invitation_mode': 'once',
               'my_did': 'WwqD8U9uUPHz8ZhVFabXVB',
               'routing_state': 'none',
               'state': 'active',
               'their_did': 'EqetcsEvB8ZSonTMy1fFVX',
               'their_label': 'Client',
               'updated_at': '2020-06-02 10:34:30.862634Z'},
  'role': None,
  'state': 'active',
  'their_did': 'EqetcsEvB8ZSonTMy1fFVX'},
 {'connection_id': '32440029-af56-47aa-bd08-04d5a7a9949d',
  'label': 'Client',
  'my_did': '2ZCYcLZsqGGrvxahUgHH2M',
  'raw_repr': {'accept': 'auto',
               'connection_id': '32440029-af56-4

In [28]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/connection-get-list"
)
connection_list = util.sendMessage(message, connections[1])
pp.pprint(connection_list['results'][-2])

{'accept': 'auto',
 'connection_id': '431acade-ed1c-4084-83cd-4084bdbcc2c0',
 'created_at': '2020-06-02 10:49:22.899715Z',
 'initiator': 'external',
 'invitation_key': 'GmEA8nDqukbRofaXDSuRCE2UyLA7heUzxnxdTHwKHnYm',
 'invitation_mode': 'once',
 'my_did': '4M6tANeLSqzAY6pU1x68Zg',
 'request_id': '00624512-b504-4d07-a10e-8b56cecba699',
 'routing_state': 'none',
 'state': 'active',
 'their_did': '2ZCYcLZsqGGrvxahUgHH2M',
 'their_label': 'ConnectionBetweenAgents',
 'updated_at': '2020-06-02 10:49:23.310499Z'}


In [14]:
message = util.buildMessage(
    "https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1/send",
    connection_id="8b217d00-aaba-4db2-bff4-8b6616345048",
    content="halo"
)
public_did = util.sendMessage(message, connections[0])
pp.pprint(public_did)


{'@id': '55c8e14b-aa56-46ee-88c0-a9d6c051e67e',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1/sent',
 'connection_id': '8b217d00-aaba-4db2-bff4-8b6616345048',
 'message': {'connection_id': '8b217d00-aaba-4db2-bff4-8b6616345048',
             'content': 'halo',
             'message_id': 'bf04f0e0-595a-4f3b-aa2f-06d8907827c0',
             'sent_time': '2020-06-02T10:49:23Z',
             'state': 'sent'},
 '~thread': {'thid': '4b404045-a89e-4397-9d01-9415681cccf5'}}


In [15]:
message = util.buildMessage(
    "https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1/get"
)
public_did = util.sendMessage(message, connections[1])
pp.pprint(public_did)

{'@id': '89f3741c-66d6-4a29-b820-3e5291a6a96a',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1/messages',
 'count': 2,
 'messages': [{'connection_id': '951c7526-af69-45df-aad7-c9f2f4173b77',
               'content': 'halo',
               'message_id': 'bf04f0e0-595a-4f3b-aa2f-06d8907827c0',
               'sent_time': '2020-06-02T10:49:23Z',
               'state': 'recv'},
              {'connection_id': '951c7526-af69-45df-aad7-c9f2f4173b77',
               'content': 'halo',
               'message_id': '7123a7dd-ed26-4162-b976-f6c688ad2369',
               'sent_time': '2020-06-02T10:35:48Z',
               'state': 'recv'}],
 'offset': 0,
 'remaining': 0,
 '~thread': {'thid': '0497985c-3454-4e7d-a662-16881d205309'}}
